In [1]:
! nvidia-smi

Mon Nov  8 07:30:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import files
files.upload()

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download -d moltean/fruits

 99% 1.27G/1.28G [00:08<00:00, 174MB/s]
100% 1.28G/1.28G [00:08<00:00, 159MB/s]


In [7]:
import zipfile

unzip = zipfile.ZipFile("fruits.zip","r")
unzip.extractall()
unzip.close()

In [8]:
# training and testing dataset

train_dir = "/content/fruits-360_dataset/fruits-360/Training"
test_dir = "/content/fruits-360_dataset/fruits-360/Test"


In [9]:
# processing the dataset
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)


In [10]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           class_mode="categorical",
                                           batch_size = 32
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (224,224),
                                         class_mode = "categorical",
                                         batch_size = 32
                                         )

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [11]:
train_data.classes

array([  0,   0,   0, ..., 130, 130, 130], dtype=int32)

In [12]:
# building a CNN model
import tensorflow as tf
from tensorflow.keras import layers


model = tf.keras.Sequential([
                             layers.Conv2D(filters= 128, kernel_size= 3, activation="relu",input_shape=(224,224,3)),
                             layers.Dropout(0.2),
                             layers.Conv2D(filters = 128, kernel_size= 3, activation="relu"),
                             layers.Dropout(0.2),
                             layers.Conv2D(filters = 64, kernel_size= 2, activation= "relu"),
                             layers.Dropout(0.2),
                             layers.Conv2D(filters= 64, kernel_size= 2, activation= "relu"),
                             layers.MaxPool2D(pool_size= 2),
                             layers.Flatten(),

                             layers.Dense(131, activation="softmax")

])


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 128)     3584      
_________________________________________________________________
dropout (Dropout)            (None, 222, 222, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 128)     147584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 220, 220, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 219, 219, 64)      32832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 219, 219, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 218, 218, 64)      1

In [14]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [15]:
# fitting the data to the model

model.fit(train_data,
          epochs = 1,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

2116/2116 [==============================] - 3602s 2s/step - loss: 0.7207 - accuracy: 0.8446 - val_loss: 0.5298 - val_accuracy: 0.8911


In [16]:
model_evaluation = model.evaluate(test_data)

709/709 [==============================] - 286s 403ms/step - loss: 0.5298 - accuracy: 0.8911


In [17]:
print(f"CNN Model Accuracy: {model_evaluation[1] *100: 0.2f}%")

CNN Model Accuracy:  89.11%


In [18]:
# Building a model in resnet with transfer learning
import tensorflow as tf

base_model = tf.keras.applications.ResNet50V2(include_top= False, weights= "imagenet")

94683136/94668760 [==============================] - 1s 0us/step


In [19]:
import tensorflow as tf
from tensorflow.keras import layers

for layer in base_model.layers[:-5]:
  base_model.trainable = False





In [20]:
inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(131, activation="softmax")(x)

resnet_model = tf.keras.Model(inputs,outputs)

In [32]:
resnet_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 131)               268419    
Total params: 23,833,219
Trainable params: 268,419
Non-trainable params: 23,564,800
_________________________________________________________________


In [21]:
# compiling the resnet model

resnet_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer= tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [22]:
# fitting the model

resnet_model.fit(train_data,
                 epochs = 1,
                 steps_per_epoch= len(train_data),
                #  validation_data = test_data,
                #  validation_steps = len(test_data)
                 )

2116/2116 [==============================] - 431s 202ms/step - loss: 0.1859 - accuracy: 0.9717


In [23]:
resent_model_evaluate = resnet_model.evaluate(test_data)

709/709 [==============================] - 144s 201ms/step - loss: 0.1391 - accuracy: 0.9635


In [25]:
print(f"Resnet Model Accuracy: {resent_model_evaluate[1] * 100: 0.2f}%")

Resnet Model Accuracy:  96.35%


In [37]:
# preparing data for efficientnet
import tensorflow as tf
train_data_eff = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode = "categorical",
                                                                     image_size = (224,224),
                                                                     batch_size = 32
                                                                     )
test_data_eff = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode = "categorical",
                                                                    image_size = (224,224),
                                                                    batch_size = 32
                                                                    )

Found 67692 files belonging to 131 classes.
Found 22688 files belonging to 131 classes.


In [26]:
# building efficientnetB3 model
import tensorflow as tf


eff_model = tf.keras.applications.EfficientNetB3(include_top= False, weights= "imagenet")



43950080/43941136 [==============================] - 1s 0us/step


In [27]:
import tensorflow as tf
from tensorflow.keras import layers

for layer in eff_model.layers[:-5]:
  eff_model.trainable = False




In [33]:
inputs = tf.keras.Input(shape=(224,224,3))

x = eff_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(131, activation="softmax")(x)

model_efficient = tf.keras.Model(inputs,outputs)

In [34]:
model_efficient.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, None, None, 1536)  10783535  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1536)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               196736    
_________________________________________________________________
dense_4 (Dense)              (None, 131)               16899     
Total params: 10,997,170
Trainable params: 213,635
Non-trainable params: 10,783,535
_________________________________________

In [35]:
# compiling the model

model_efficient.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer= tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics= ["accuracy"]
)

In [38]:
# fitting the model

model_efficient.fit(
    train_data_eff,
    epochs = 1,
    steps_per_epoch = len(train_data_eff),
#     validation_data = test_data_eff,
#     validation_steps = len(test_data_eff)
)

2116/2116 [==============================] - 535s 250ms/step - loss: 0.1930 - accuracy: 0.9545


In [40]:
model_efficient_evaluation = model_efficient.evaluate(test_data_eff)

709/709 [==============================] - 178s 247ms/step - loss: 0.1157 - accuracy: 0.9716


In [41]:
print(f"Efficient Model Accuracy: {model_efficient_evaluation[1]*100 : 0.2f}%")

Efficient Model Accuracy:  97.16%
